### LSTM with stock price data - sentiment and stocks

Note: The same process can be conducted using another data format - instead of using stock prices one can also use stock returns, for this use the file "LSTM_return_170621.xlsx". However, also this model leads to no satisfactory results.

In [1]:
# Import modules and packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from tensorflow.keras.utils import to_categorical

%matplotlib inline

In [2]:
from numpy.random import seed
seed(1)
tf.random.set_seed(2)

In [3]:
df = pd.read_excel('LSTM_price_170621.xlsx', header=0)

In [4]:
df.set_index('publication_date', inplace = True)

In [5]:
dummies = pd.get_dummies(df.sentiment, drop_first = True)
dummies

,0,1
publication_date,,
44257,0,1
44257,0,1
44201,1,0
42710,1,0
44174,0,1
...,...,...
42580,0,1
44041,0,0
44041,0,1


In [6]:
df =pd.concat([df,dummies],axis=1)
df

,company_ticker,market_cap_mio_chf,mid_or_small,document_name,document_type,source,original_text,flesch_score,wiener_score,sentiment,...,t-13,t-14,t-15,t-16,t-17,t-18,t-19,t-20,0,1
publication_date,,,,,,,,,,,,,,,,,,,,,
44257,ARBN,1198,small_cap,ARBN_02.03.2021.txt,news,cash.ch,Der Bauzulieferer Arbonia hat im vergangenen G...,43.880769,11.839886,1,...,14.491424,14.180199,14.024586,14.082940,14.063489,14.258004,14.180199,14.044037,0,1
44257,ARBN,1198,small_cap,ARBN_02.03.2021_2.txt,news,cash.ch,Die Aktien von Arbonia legen am Dienstag klar ...,48.471429,10.630978,1,...,14.491424,14.180199,14.024586,14.082940,14.063489,14.258004,14.180199,14.044037,0,1
44201,ARBN,1198,small_cap,ARBN_05.01.2021.txt,news,cash.ch,Der Verkauf des Fenstergeschäfts hat der Arbon...,30.957143,13.276634,0,...,13.207620,13.149265,12.740782,12.371202,12.487911,12.410106,12.176686,12.293396,1,0
42710,ARBN,1198,small_cap,ARBN_06.12.2016.txt,news,cash.ch,Der Bauzulieferer AFG Arbonia Forster hat im R...,39.566667,16.064963,0,...,14.117880,14.117880,14.071440,13.978559,13.699916,13.746356,13.374833,13.281953,1,0
44174,ARBN,1198,small_cap,ARBN_09.12.2020.txt,news,cash.ch,Beim Bauausrüster Arbonia hat sich im zweiten ...,53.879167,10.673042,1,...,12.449008,12.429556,12.273944,11.884912,12.215590,12.157234,12.001622,11.865460,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42580,ZEHN,972,small_cap,zehn_29.07.2016.txt,quarterly_earnings_report,company,Umsatzsteigerung und Ergebnisverbesserung. Die...,46.597119,11.132120,1,...,33.052650,32.917191,31.517420,31.246490,31.156187,32.059265,33.775120,34.316959,0,1
44041,ZEHN,972,small_cap,ZEHN_29.07.2020 (2).txt,news,cash.ch,Zehnder übertrifft Erwartungen deutlich. Die a...,58.866667,9.486783,-1,...,35.717522,35.373138,35.865116,36.455486,36.160301,36.455486,36.652279,36.504684,0,0
44041,ZEHN,972,small_cap,ZEHN_29.07.2020.txt,news,cash.ch,Zehnder-Aktien legen kräftig zu - Halbjahresza...,38.060164,12.875756,1,...,35.717522,35.373138,35.865116,36.455486,36.160301,36.455486,36.652279,36.504684,0,1


In [7]:
#defining output variable
y = df['t+0']
#y = tf.keras.utils.to_categorical(y, 3)
y = y.replace(-1,0)

In [8]:
#defining features
x1 = df.iloc[:,12:-2]
x2 = df.iloc[:,-2:]

In [9]:
x1.head()

,t-1,t-2,t-3,t-4,t-5,t-6,t-7,t-8,t-9,t-10,t-11,t-12,t-13,t-14,t-15,t-16,t-17,t-18,t-19,t-20
publication_date,,,,,,,,,,,,,,,,,,,,
44257,14.880455,14.841552,14.841552,14.666488,14.394166,14.355263,14.102392,13.771715,14.005134,14.238553,14.219101,14.296907,14.491424,14.180199,14.024586,14.082940,14.063489,14.258004,14.180199,14.044037
44257,14.880455,14.841552,14.841552,14.666488,14.394166,14.355263,14.102392,13.771715,14.005134,14.238553,14.219101,14.296907,14.491424,14.180199,14.024586,14.082940,14.063489,14.258004,14.180199,14.044037
44201,13.810618,13.771715,13.771715,13.518845,13.557748,13.363232,13.538297,13.732812,13.518845,13.518845,13.616102,13.479941,13.207620,13.149265,12.740782,12.371202,12.487911,12.410106,12.176686,12.293396
42710,14.396523,14.350081,14.582283,14.489403,14.350081,14.350081,14.489403,14.303641,14.257201,14.071440,14.257201,14.257201,14.117880,14.117880,14.071440,13.978559,13.699916,13.746356,13.374833,13.281953
44174,12.371202,12.487911,12.410106,12.176686,12.293396,12.312847,12.507362,12.740782,12.410106,12.351750,12.351750,12.449008,12.449008,12.429556,12.273944,11.884912,12.215590,12.157234,12.001622,11.865460


In [10]:
x2.head()

,0,1
publication_date,,
44257,0,1
44257,0,1
44201,1,0
42710,1,0
44174,0,1


In [11]:
#turn pd into arrays
x1 = x1.values
x2 = x2.values

In [12]:
print(x1.shape)
print(x2.shape)

(662, 20)
(662, 2)


In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
# define sequences

# pad sequence
padded = pad_sequences(x2, maxlen=20, padding = 'post')
print(padded)

[[0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [14]:
#reshape and combine for feature scaling
sub1 = x1.reshape(-1,1)
sub2 = padded.reshape(-1,1)
subh = np.hstack((sub1, sub2))
subh.shape

(13240, 2)

In [15]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sub1_scaled = sc.fit_transform(sub1)

padded_scaled = sc.fit_transform(padded)

In [16]:
#reshape into correct format
sub1_scaled = sub1_scaled.reshape(662,20)

In [17]:
padded_scaled.shape

(662, 20)

In [18]:
#stack both features 
subh_scaled = np.hstack((sub1_scaled, padded_scaled))

In [19]:
#rearrange array into correct format for train test split
split = np.array(np.split(subh, 662))

In [20]:
split[0]

array([[14.88045502,  0.        ],
       [14.84155178,  1.        ],
       [14.84155178,  0.        ],
       [14.66648769,  0.        ],
       [14.39416599,  0.        ],
       [14.35526276,  0.        ],
       [14.1023922 ,  0.        ],
       [13.77171516,  0.        ],
       [14.00513363,  0.        ],
       [14.23855305,  0.        ],
       [14.21910095,  0.        ],
       [14.29690742,  0.        ],
       [14.49142361,  0.        ],
       [14.18019867,  0.        ],
       [14.02458572,  0.        ],
       [14.0829401 ,  0.        ],
       [14.06348896,  0.        ],
       [14.25800419,  0.        ],
       [14.18019867,  0.        ],
       [14.04403687,  0.        ]])

In [21]:
#y as array
y = y.values

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(subh_scaled, y, random_state = 2)

In [23]:
X_train.shape

(496, 40)

In [24]:
#reshape into usable format for LSTM
X_train = X_train.reshape(496,20,2)

In [25]:
X_test.shape

(166, 40)

In [26]:
#reshape into usable format for LSTM
X_test = X_test.reshape(166,20,2)

In [27]:
np_split = np.asarray(split)
np_split.shape

(662, 20, 2)

In [29]:
X_train.shape

(496, 20, 2)

In [30]:
y_train.shape

(496,)

In [31]:
# Import Libraries and packages from Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam

# Initializing the Neural Network based on LSTM
model = Sequential()

# Adding 1st LSTM layer
model.add(LSTM(units=512, return_sequences=True, input_shape=(X_train.shape[1],2)))
# Adding 2nd LSTM layer
model.add(LSTM(units=512, return_sequences=True))
# Adding Dropout
model.add(Dropout(0.5))

# Output layer
model.add(Dense(units=1))

# Compiling the Neural Network
model.compile(optimizer = Adam(learning_rate=0.01), loss='mean_squared_error', metrics = ['accuracy'])



In [32]:
%%time
# es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=10, verbose=1)
# rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1)
# mcp = ModelCheckpoint(filepath='weights.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)

# tb = TensorBoard('logs')

history = model.fit(X_train, y_train, shuffle=False, epochs=20, validation_split=0.2, verbose=1, batch_size=256, )

Epoch 1/20
2/2 [==============================] - 2s 1s/step - loss: 105306.6094 - accuracy: 0.0000e+00 - val_loss: 124194.1406 - val_accuracy: 0.0000e+00
Epoch 2/20
2/2 [==============================] - 2s 843ms/step - loss: 100041.7578 - accuracy: 0.0000e+00 - val_loss: 121306.2578 - val_accuracy: 0.0000e+00
Epoch 3/20
2/2 [==============================] - 2s 772ms/step - loss: 97519.3828 - accuracy: 0.0000e+00 - val_loss: 118746.9766 - val_accuracy: 0.0000e+00
Epoch 4/20
2/2 [==============================] - 2s 754ms/step - loss: 95321.7969 - accuracy: 0.0000e+00 - val_loss: 116277.2578 - val_accuracy: 0.0000e+00
Epoch 5/20
2/2 [==============================] - 1s 744ms/step - loss: 93051.8750 - accuracy: 0.0000e+00 - val_loss: 113672.9922 - val_accuracy: 0.0000e+00
Epoch 6/20
2/2 [==============================] - 1s 740ms/step - loss: 90800.3672 - accuracy: 0.0000e+00 - val_loss: 111032.9375 - val_accuracy: 0.0000e+00
Epoch 7/20
2/2 [==============================] - 2s 852ms/

In [33]:
test_loss, test_acc = model.evaluate(X_test, y_test)

6/6 [==============================] - 1s 126ms/step - loss: 86197.4609 - accuracy: 0.0000e+00


In [34]:
unique, counts = np.unique(y_train, return_counts=True)

print (np.asarray((unique, counts)).T)
print('Accuracy if only prediction ones:', (np.asarray((unique, counts)).T)[1][1] / ((np.asarray((unique, counts)).T)[0][1] + (np.asarray((unique, counts)).T)[1][1] ) )

[[5.11494255e+00 1.00000000e+00]
 [6.44555616e+00 1.00000000e+00]
 [6.67782879e+00 2.00000000e+00]
 [6.77460909e+00 2.00000000e+00]
 [6.78917027e+00 1.00000000e+00]
 [7.06324339e+00 1.00000000e+00]
 [7.27902555e+00 1.00000000e+00]
 [7.57788372e+00 1.00000000e+00]
 [7.64563036e+00 2.00000000e+00]
 [7.73251200e+00 2.00000000e+00]
 [8.36909676e+00 1.00000000e+00]
 [8.49625206e+00 4.00000000e+00]
 [8.80947685e+00 1.00000000e+00]
 [8.89439201e+00 3.00000000e+00]
 [9.49091148e+00 3.00000000e+00]
 [9.92030334e+00 1.00000000e+00]
 [1.00237789e+01 1.00000000e+00]
 [1.01232014e+01 2.00000000e+00]
 [1.01764660e+01 1.00000000e+00]
 [1.02653151e+01 2.00000000e+00]
 [1.03482380e+01 2.00000000e+00]
 [1.04968777e+01 2.00000000e+00]
 [1.07600002e+01 1.00000000e+00]
 [1.09866657e+01 3.00000000e+00]
 [1.11457529e+01 1.00000000e+00]
 [1.11799049e+01 1.00000000e+00]
 [1.13522320e+01 3.00000000e+00]
 [1.23183794e+01 1.00000000e+00]
 [1.23586359e+01 1.00000000e+00]
 [1.26565313e+01 1.00000000e+00]
 [1.274078

In [35]:
unique, counts = np.unique(y_test, return_counts=True)

print (np.asarray((unique, counts)).T)
print('Accuracy if only prediction ones:', (np.asarray((unique, counts)).T)[1][1] / ((np.asarray((unique, counts)).T)[0][1] + (np.asarray((unique, counts)).T)[1][1] ) )

[[6.67782879e+00 1.00000000e+00]
 [6.77460909e+00 1.00000000e+00]
 [7.57788372e+00 1.00000000e+00]
 [7.64563036e+00 1.00000000e+00]
 [7.73251200e+00 1.00000000e+00]
 [8.32458115e+00 1.00000000e+00]
 [9.62988853e+00 1.00000000e+00]
 [1.01232014e+01 1.00000000e+00]
 [1.02399998e+01 2.00000000e+00]
 [1.02653151e+01 1.00000000e+00]
 [1.04812880e+01 1.00000000e+00]
 [1.11469259e+01 1.00000000e+00]
 [1.20208702e+01 1.00000000e+00]
 [1.27711449e+01 1.00000000e+00]
 [1.28819647e+01 1.00000000e+00]
 [1.35379086e+01 1.00000000e+00]
 [1.40325937e+01 1.00000000e+00]
 [1.43965225e+01 1.00000000e+00]
 [1.54182110e+01 1.00000000e+00]
 [1.54250984e+01 2.00000000e+00]
 [1.56039715e+01 1.00000000e+00]
 [1.58826151e+01 1.00000000e+00]
 [1.64399014e+01 1.00000000e+00]
 [2.58114891e+01 1.00000000e+00]
 [2.59046669e+01 1.00000000e+00]
 [2.75819492e+01 1.00000000e+00]
 [2.76590900e+01 1.00000000e+00]
 [2.79624176e+01 1.00000000e+00]
 [2.80643711e+01 1.00000000e+00]
 [2.81166267e+01 1.00000000e+00]
 [2.953065

In [36]:
y_pred = (model.predict(X_test))

In [37]:
y_pred

array([[[ 9.88928146e+01],
        [ 1.24248901e+02],
        [ 1.28214310e+02],
        ...,
        [ 1.28854782e+02],
        [ 1.28854828e+02],
        [ 1.28854858e+02]],

       [[ 1.88180237e+01],
        [ 1.05940819e+02],
        [ 1.24959419e+02],
        ...,
        [ 1.28850937e+02],
        [ 1.28851837e+02],
        [ 1.28852539e+02]],

       [[-5.78618050e-02],
        [ 1.90846901e+01],
        [ 2.79657040e+01],
        ...,
        [ 2.97460155e+01],
        [ 2.97463112e+01],
        [ 2.97466145e+01]],

       ...,

       [[ 7.40726013e+01],
        [ 1.21925606e+02],
        [ 1.27865852e+02],
        ...,
        [ 1.28853882e+02],
        [ 1.28854126e+02],
        [ 1.28854324e+02]],

       [[ 8.70874405e+01],
        [ 1.23423172e+02],
        [ 1.28079514e+02],
        ...,
        [ 1.28854187e+02],
        [ 1.28854370e+02],
        [ 1.28854507e+02]],

       [[-6.02112889e-01],
        [ 2.18800373e+01],
        [ 2.84079971e+01],
        ...,
        

In [41]:
y_test[0]

717.957763671875

In [42]:
model.predict(X_test)[0]

array([[ 98.892815],
       [124.2489  ],
       [128.21431 ],
       [128.76192 ],
       [128.8376  ],
       [128.84894 ],
       [128.85129 ],
       [128.85231 ],
       [128.85292 ],
       [128.85338 ],
       [128.85374 ],
       [128.85402 ],
       [128.85425 ],
       [128.85442 ],
       [128.85454 ],
       [128.85464 ],
       [128.85472 ],
       [128.85478 ],
       [128.85483 ],
       [128.85486 ]], dtype=float32)

Result: Model did not converge, meaning that its loss is too high to create usable results.

### LSTM with only stock price data - no sentiment
This model uses the same approach as above but with no sentiment data

In [44]:
df = pd.read_excel('LSTM_price_170621.xlsx', header=0)

In [45]:
df.set_index('publication_date', inplace = True)

In [46]:
y = df['t+0']
y = y.replace(-1,0)

In [47]:
x1 = df.iloc[:,12:]

x1.head()


,t-1,t-2,t-3,t-4,t-5,t-6,t-7,t-8,t-9,t-10,t-11,t-12,t-13,t-14,t-15,t-16,t-17,t-18,t-19,t-20
publication_date,,,,,,,,,,,,,,,,,,,,
44257,14.880455,14.841552,14.841552,14.666488,14.394166,14.355263,14.102392,13.771715,14.005134,14.238553,14.219101,14.296907,14.491424,14.180199,14.024586,14.082940,14.063489,14.258004,14.180199,14.044037
44257,14.880455,14.841552,14.841552,14.666488,14.394166,14.355263,14.102392,13.771715,14.005134,14.238553,14.219101,14.296907,14.491424,14.180199,14.024586,14.082940,14.063489,14.258004,14.180199,14.044037
44201,13.810618,13.771715,13.771715,13.518845,13.557748,13.363232,13.538297,13.732812,13.518845,13.518845,13.616102,13.479941,13.207620,13.149265,12.740782,12.371202,12.487911,12.410106,12.176686,12.293396
42710,14.396523,14.350081,14.582283,14.489403,14.350081,14.350081,14.489403,14.303641,14.257201,14.071440,14.257201,14.257201,14.117880,14.117880,14.071440,13.978559,13.699916,13.746356,13.374833,13.281953
44174,12.371202,12.487911,12.410106,12.176686,12.293396,12.312847,12.507362,12.740782,12.410106,12.351750,12.351750,12.449008,12.449008,12.429556,12.273944,11.884912,12.215590,12.157234,12.001622,11.865460


In [48]:
x1 = x1.values


print(x1.shape)

(662, 20)


In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x1, y, random_state = 2, shuffle = False)

X_train.shape

y_train.shape

(496,)

In [50]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
training_set_scaled = sc.fit_transform(X_train)

sc_predict = StandardScaler()
sc_predict.fit_transform(X_test)

array([[-0.25250002, -0.24744124, -0.26022036, ..., -0.24326984,
        -0.24001385, -0.24083201],
       [-0.23862474, -0.25086511, -0.2497268 , ..., -0.17538867,
        -0.14894703, -0.1570707 ],
       [-0.23862474, -0.25086511, -0.2497268 , ..., -0.17538867,
        -0.14894703, -0.1570707 ],
       ...,
       [-0.87445612, -0.86798506, -0.86373556, ..., -0.85046945,
        -0.84164413, -0.8439267 ],
       [-0.87445612, -0.86798506, -0.86373556, ..., -0.85046945,
        -0.84164413, -0.8439267 ],
       [-0.91563299, -0.91594939, -0.91803649, ..., -0.89738993,
        -0.88852504, -0.89399713]])

In [51]:
X_train = X_train.reshape(496, 20, 1)

In [53]:
X_train.shape

(496, 20, 1)

In [54]:
X_test = X_test.reshape(166, 20, 1)

In [55]:
X_test.shape

(166, 20, 1)

In [56]:
sub1 = x1.reshape(-1,1)

In [57]:
sub1

array([[14.88045502],
       [14.84155178],
       [14.84155178],
       ...,
       [31.68459892],
       [31.87070084],
       [31.40543938]])

In [58]:
split = np.split(sub1, 662)

split[2]

array([[13.8106184 ],
       [13.77171516],
       [13.77171516],
       [13.5188446 ],
       [13.55774784],
       [13.36323166],
       [13.5382967 ],
       [13.73281193],
       [13.5188446 ],
       [13.5188446 ],
       [13.61610222],
       [13.47994137],
       [13.20761967],
       [13.14926529],
       [12.74078178],
       [12.37120247],
       [12.48791122],
       [12.41010571],
       [12.17668629],
       [12.293396  ]])

In [59]:
daten = np.array(split)

In [60]:
np.array(daten).reshape(662,20)

array([[14.88045502, 14.84155178, 14.84155178, ..., 14.25800419,
        14.18019867, 14.04403687],
       [14.88045502, 14.84155178, 14.84155178, ..., 14.25800419,
        14.18019867, 14.04403687],
       [13.8106184 , 13.77171516, 13.77171516, ..., 12.41010571,
        12.17668629, 12.293396  ],
       ...,
       [36.50468445, 37.24264908, 37.63623047, ..., 36.4554863 ,
        36.6522789 , 36.50468445],
       [36.50468445, 37.24264908, 37.63623047, ..., 36.4554863 ,
        36.6522789 , 36.50468445],
       [32.52207565, 32.61513138, 32.38249588, ..., 31.68459892,
        31.87070084, 31.40543938]])

In [61]:
daten.shape

(662, 20, 1)

In [62]:
y_train = y_train.values
y_test = y_test.values

In [63]:
y_train[0]

15.4250984191894

In [64]:
np_split = np.asarray(split)
np_split.shape

(662, 20, 1)

In [65]:
np_split[0][1][0]

14.8415517807006

In [66]:
# Import Libraries and packages from Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam

# Initializing the Neural Network based on LSTM
model = Sequential()

# Adding 1st LSTM layer
model.add(LSTM(units=22, return_sequences=True, input_shape=(20,1)))
# Adding 2nd LSTM layer
model.add(LSTM(units=128, return_sequences=False))
# Adding Dropout
model.add(Dropout(0.2))

# Output layer
model.add(Dense(units=1, activation = 'linear'))

# Compiling the Neural Network
model.compile(optimizer = Adam(learning_rate=0.01), loss='mean_squared_error', metrics = ['accuracy'])



In [67]:
%%time
# es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=10, verbose=1)
# rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1)
# mcp = ModelCheckpoint(filepath='weights.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)

# tb = TensorBoard('logs')

history = model.fit(X_train, y_train, shuffle=False, epochs=1000, validation_split=0.2, verbose=1, batch_size=256, )


Epoch 1/1000
2/2 [==============================] - 1s 448ms/step - loss: 107083.8750 - accuracy: 0.0000e+00 - val_loss: 302215.5312 - val_accuracy: 0.0000e+00
Epoch 2/1000
2/2 [==============================] - 0s 47ms/step - loss: 104486.3047 - accuracy: 0.0000e+00 - val_loss: 298459.1250 - val_accuracy: 0.0000e+00
Epoch 3/1000
2/2 [==============================] - 0s 35ms/step - loss: 102439.5781 - accuracy: 0.0000e+00 - val_loss: 295078.9062 - val_accuracy: 0.0000e+00
Epoch 4/1000
2/2 [==============================] - 0s 33ms/step - loss: 100766.5859 - accuracy: 0.0000e+00 - val_loss: 292960.9062 - val_accuracy: 0.0000e+00
Epoch 5/1000
2/2 [==============================] - 0s 42ms/step - loss: 99618.0781 - accuracy: 0.0000e+00 - val_loss: 291189.6875 - val_accuracy: 0.0000e+00
Epoch 6/1000
2/2 [==============================] - 0s 28ms/step - loss: 98711.7812 - accuracy: 0.0000e+00 - val_loss: 289471.5625 - val_accuracy: 0.0000e+00
Epoch 7/1000
2/2 [=============================

2/2 [==============================] - 0s 30ms/step - loss: 49183.4648 - accuracy: 0.0000e+00 - val_loss: 190194.5156 - val_accuracy: 0.0000e+00
Epoch 104/1000
2/2 [==============================] - 0s 35ms/step - loss: 48861.1680 - accuracy: 0.0000e+00 - val_loss: 189361.1250 - val_accuracy: 0.0000e+00
Epoch 105/1000
2/2 [==============================] - 0s 34ms/step - loss: 48475.2461 - accuracy: 0.0000e+00 - val_loss: 188427.1562 - val_accuracy: 0.0000e+00
Epoch 106/1000
2/2 [==============================] - 0s 36ms/step - loss: 48118.9844 - accuracy: 0.0000e+00 - val_loss: 187398.2344 - val_accuracy: 0.0000e+00
Epoch 107/1000
2/2 [==============================] - 0s 54ms/step - loss: 47723.0625 - accuracy: 0.0000e+00 - val_loss: 186299.3438 - val_accuracy: 0.0000e+00
Epoch 108/1000
2/2 [==============================] - 0s 32ms/step - loss: 47401.2930 - accuracy: 0.0000e+00 - val_loss: 185197.6406 - val_accuracy: 0.0000e+00
Epoch 109/1000
2/2 [==============================] - 0

2/2 [==============================] - 0s 25ms/step - loss: 27051.3105 - accuracy: 0.0000e+00 - val_loss: 125592.1016 - val_accuracy: 0.0000e+00
Epoch 206/1000
2/2 [==============================] - 0s 30ms/step - loss: 27514.8027 - accuracy: 0.0000e+00 - val_loss: 125093.1719 - val_accuracy: 0.0000e+00
Epoch 207/1000
2/2 [==============================] - 0s 30ms/step - loss: 27585.9805 - accuracy: 0.0000e+00 - val_loss: 123844.1172 - val_accuracy: 0.0000e+00
Epoch 208/1000
2/2 [==============================] - 0s 47ms/step - loss: 25539.3945 - accuracy: 0.0000e+00 - val_loss: 127867.9375 - val_accuracy: 0.0000e+00
Epoch 209/1000
2/2 [==============================] - 0s 33ms/step - loss: 26609.4023 - accuracy: 0.0000e+00 - val_loss: 130512.6172 - val_accuracy: 0.0000e+00
Epoch 210/1000
2/2 [==============================] - 0s 31ms/step - loss: 29753.0508 - accuracy: 0.0000e+00 - val_loss: 138177.4062 - val_accuracy: 0.0000e+00
Epoch 211/1000
2/2 [==============================] - 0

2/2 [==============================] - 0s 34ms/step - loss: 25786.8730 - accuracy: 0.0000e+00 - val_loss: 116960.2812 - val_accuracy: 0.0000e+00
Epoch 308/1000
2/2 [==============================] - 0s 33ms/step - loss: 25684.9648 - accuracy: 0.0000e+00 - val_loss: 116588.9062 - val_accuracy: 0.0000e+00
Epoch 309/1000
2/2 [==============================] - 0s 56ms/step - loss: 25022.3359 - accuracy: 0.0000e+00 - val_loss: 116329.9766 - val_accuracy: 0.0000e+00
Epoch 310/1000
2/2 [==============================] - 0s 34ms/step - loss: 24038.1465 - accuracy: 0.0000e+00 - val_loss: 116136.4531 - val_accuracy: 0.0000e+00
Epoch 311/1000
2/2 [==============================] - 0s 35ms/step - loss: 24587.4121 - accuracy: 0.0000e+00 - val_loss: 115794.9922 - val_accuracy: 0.0000e+00
Epoch 312/1000
2/2 [==============================] - 0s 30ms/step - loss: 24950.0508 - accuracy: 0.0000e+00 - val_loss: 114945.0000 - val_accuracy: 0.0000e+00
Epoch 313/1000
2/2 [==============================] - 0

2/2 [==============================] - 0s 31ms/step - loss: 11592.1719 - accuracy: 0.0000e+00 - val_loss: 70396.1016 - val_accuracy: 0.0000e+00
Epoch 410/1000
2/2 [==============================] - 0s 47ms/step - loss: 11355.6924 - accuracy: 0.0000e+00 - val_loss: 70102.2812 - val_accuracy: 0.0000e+00
Epoch 411/1000
2/2 [==============================] - 0s 51ms/step - loss: 11487.7852 - accuracy: 0.0000e+00 - val_loss: 69777.3203 - val_accuracy: 0.0000e+00
Epoch 412/1000
2/2 [==============================] - 0s 30ms/step - loss: 11605.6094 - accuracy: 0.0000e+00 - val_loss: 69460.6953 - val_accuracy: 0.0000e+00
Epoch 413/1000
2/2 [==============================] - 0s 33ms/step - loss: 11184.9941 - accuracy: 0.0000e+00 - val_loss: 69157.3984 - val_accuracy: 0.0000e+00
Epoch 414/1000
2/2 [==============================] - 0s 31ms/step - loss: 11241.5078 - accuracy: 0.0000e+00 - val_loss: 68857.3672 - val_accuracy: 0.0000e+00
Epoch 415/1000
2/2 [==============================] - 0s 31ms

...

Epoch 720/1000
2/2 [==============================] - 0s 25ms/step - loss: 3986.5068 - accuracy: 0.0000e+00 - val_loss: 37219.1406 - val_accuracy: 0.0000e+00
Epoch 721/1000
2/2 [==============================] - 0s 31ms/step - loss: 4294.7827 - accuracy: 0.0000e+00 - val_loss: 36986.8867 - val_accuracy: 0.0000e+00
Epoch 722/1000
2/2 [==============================] - 0s 36ms/step - loss: 4133.6851 - accuracy: 0.0000e+00 - val_loss: 36685.9883 - val_accuracy: 0.0000e+00
Epoch 723/1000
2/2 [==============================] - 0s 50ms/step - loss: 4038.4761 - accuracy: 0.0000e+00 - val_loss: 36307.7148 - val_accuracy: 0.0000e+00
Epoch 724/1000
2/2 [==============================] - 0s 53ms/step - loss: 3678.3047 - accuracy: 0.0000e+00 - val_loss: 35995.0469 - val_accuracy: 0.0000e+00
Epoch 725/1000
2/2 [==============================] - 0s 26ms/step - loss: 4330.8970 - accuracy: 0.0000e+00 - val_loss: 35741.0781 - val_accuracy: 0.0000e+00
Epoch 726/1000
2/2 [==============================] 

Epoch 772/1000
2/2 [==============================] - 0s 45ms/step - loss: 3094.5483 - accuracy: 0.0000e+00 - val_loss: 30513.3379 - val_accuracy: 0.0000e+00
Epoch 773/1000
2/2 [==============================] - 0s 28ms/step - loss: 3065.9221 - accuracy: 0.0000e+00 - val_loss: 30432.8203 - val_accuracy: 0.0000e+00
Epoch 774/1000
2/2 [==============================] - 0s 32ms/step - loss: 2890.1150 - accuracy: 0.0000e+00 - val_loss: 30416.6504 - val_accuracy: 0.0000e+00
Epoch 775/1000
2/2 [==============================] - 0s 31ms/step - loss: 3038.4431 - accuracy: 0.0000e+00 - val_loss: 30288.3008 - val_accuracy: 0.0000e+00
Epoch 776/1000
2/2 [==============================] - 0s 32ms/step - loss: 3086.7886 - accuracy: 0.0000e+00 - val_loss: 30183.0195 - val_accuracy: 0.0000e+00
Epoch 777/1000
2/2 [==============================] - 0s 29ms/step - loss: 3040.5891 - accuracy: 0.0000e+00 - val_loss: 30117.4609 - val_accuracy: 0.0000e+00
Epoch 778/1000
2/2 [==============================] 

Epoch 824/1000
2/2 [==============================] - 0s 42ms/step - loss: 2506.1335 - accuracy: 0.0000e+00 - val_loss: 26633.0234 - val_accuracy: 0.0000e+00
Epoch 825/1000
2/2 [==============================] - 0s 32ms/step - loss: 2381.8650 - accuracy: 0.0000e+00 - val_loss: 26579.2852 - val_accuracy: 0.0000e+00
Epoch 826/1000
2/2 [==============================] - 0s 28ms/step - loss: 2755.8069 - accuracy: 0.0000e+00 - val_loss: 26507.0820 - val_accuracy: 0.0000e+00
Epoch 827/1000
2/2 [==============================] - 0s 30ms/step - loss: 2628.9531 - accuracy: 0.0000e+00 - val_loss: 26379.4375 - val_accuracy: 0.0000e+00
Epoch 828/1000
2/2 [==============================] - 0s 30ms/step - loss: 2439.5659 - accuracy: 0.0000e+00 - val_loss: 26267.7383 - val_accuracy: 0.0000e+00
Epoch 829/1000
2/2 [==============================] - 0s 36ms/step - loss: 2474.9353 - accuracy: 0.0000e+00 - val_loss: 26219.7500 - val_accuracy: 0.0000e+00
Epoch 830/1000
2/2 [==============================] 

Epoch 876/1000
2/2 [==============================] - 0s 28ms/step - loss: 2972.4331 - accuracy: 0.0000e+00 - val_loss: 24341.9766 - val_accuracy: 0.0000e+00
Epoch 877/1000
2/2 [==============================] - 0s 25ms/step - loss: 2349.5486 - accuracy: 0.0000e+00 - val_loss: 24308.1699 - val_accuracy: 0.0000e+00
Epoch 878/1000
2/2 [==============================] - 0s 31ms/step - loss: 2706.1907 - accuracy: 0.0000e+00 - val_loss: 24221.7500 - val_accuracy: 0.0000e+00
Epoch 879/1000
2/2 [==============================] - 0s 34ms/step - loss: 2646.4158 - accuracy: 0.0000e+00 - val_loss: 30362.2852 - val_accuracy: 0.0000e+00
Epoch 880/1000
2/2 [==============================] - 0s 39ms/step - loss: 28854.0430 - accuracy: 0.0000e+00 - val_loss: 71538.7969 - val_accuracy: 0.0000e+00
Epoch 881/1000
2/2 [==============================] - 0s 75ms/step - loss: 30872.0801 - accuracy: 0.0000e+00 - val_loss: 152899.0156 - val_accuracy: 0.0000e+00
Epoch 882/1000
2/2 [=============================

Epoch 979/1000
2/2 [==============================] - 0s 30ms/step - loss: 3458.5269 - accuracy: 0.0000e+00 - val_loss: 29780.2773 - val_accuracy: 0.0000e+00
Epoch 980/1000
2/2 [==============================] - 0s 30ms/step - loss: 2975.3210 - accuracy: 0.0000e+00 - val_loss: 29182.8379 - val_accuracy: 0.0000e+00
Epoch 981/1000
2/2 [==============================] - 0s 31ms/step - loss: 3059.2109 - accuracy: 0.0000e+00 - val_loss: 28739.7930 - val_accuracy: 0.0000e+00
Epoch 982/1000
2/2 [==============================] - 0s 30ms/step - loss: 3137.0730 - accuracy: 0.0000e+00 - val_loss: 28305.2891 - val_accuracy: 0.0000e+00
Epoch 983/1000
2/2 [==============================] - 0s 36ms/step - loss: 2935.8083 - accuracy: 0.0000e+00 - val_loss: 27573.1055 - val_accuracy: 0.0000e+00
Epoch 984/1000
2/2 [==============================] - 0s 26ms/step - loss: 2428.7061 - accuracy: 0.0000e+00 - val_loss: 27521.3047 - val_accuracy: 0.0000e+00
Epoch 985/1000
2/2 [==============================] 

In [68]:
test_loss, test_acc = model.evaluate(X_test, y_test)

6/6 [==============================] - 0s 4ms/step - loss: 274.3628 - accuracy: 0.0000e+00


In [69]:
unique, counts = np.unique(y_train, return_counts=True)

print (np.asarray((unique, counts)).T)

[[5.11494255e+00 1.00000000e+00]
 [6.44555616e+00 1.00000000e+00]
 [6.67782879e+00 3.00000000e+00]
 [6.77460909e+00 3.00000000e+00]
 [6.78917027e+00 1.00000000e+00]
 [7.06324339e+00 1.00000000e+00]
 [7.27902555e+00 1.00000000e+00]
 [7.57788372e+00 2.00000000e+00]
 [7.64563036e+00 3.00000000e+00]
 [7.73251200e+00 3.00000000e+00]
 [8.32458115e+00 1.00000000e+00]
 [8.36909676e+00 1.00000000e+00]
 [8.49625206e+00 4.00000000e+00]
 [8.80947685e+00 1.00000000e+00]
 [8.89439201e+00 3.00000000e+00]
 [9.49091148e+00 3.00000000e+00]
 [9.62988853e+00 1.00000000e+00]
 [9.92030334e+00 1.00000000e+00]
 [1.00237789e+01 1.00000000e+00]
 [1.01232014e+01 3.00000000e+00]
 [1.01764660e+01 1.00000000e+00]
 [1.02399998e+01 2.00000000e+00]
 [1.02653151e+01 3.00000000e+00]
 [1.03482380e+01 2.00000000e+00]
 [1.04812880e+01 1.00000000e+00]
 [1.04968777e+01 2.00000000e+00]
 [1.07600002e+01 1.00000000e+00]
 [1.09866657e+01 3.00000000e+00]
 [1.11457529e+01 1.00000000e+00]
 [1.11469259e+01 1.00000000e+00]
 [1.117990

In [70]:
unique, counts = np.unique(y_test, return_counts=True)

print (np.asarray((unique, counts)).T)

[[ 29.53065109   2.        ]
 [ 31.63807106   3.        ]
 [ 31.65287971   4.        ]
 [ 32.01028442   1.        ]
 [ 32.05926514   3.        ]
 [ 32.21328735   1.        ]
 [ 32.42902374   1.        ]
 [ 32.72322845   1.        ]
 [ 33.36873245   1.        ]
 [ 33.86542511   1.        ]
 [ 33.90031815   1.        ]
 [ 34.09399414   1.        ]
 [ 34.67819595   1.        ]
 [ 34.91379929   2.        ]
 [ 36.48435211   1.        ]
 [ 36.80252075   2.        ]
 [ 38.15504074   1.        ]
 [ 38.22660446   3.        ]
 [ 38.71011353   1.        ]
 [ 38.84412003   1.        ]
 [ 39.64454269   2.        ]
 [ 39.97412872   1.        ]
 [ 40.49205017   2.        ]
 [ 40.82164001   2.        ]
 [ 40.85033417   3.        ]
 [ 41.10414124   1.        ]
 [ 41.54822922   3.        ]
 [ 42.16241837   1.        ]
 [ 43.1117897    1.        ]
 [ 43.1955719    1.        ]
 [ 43.40112686   1.        ]
 [ 43.88336563   2.        ]
 [ 44.02803421   2.        ]
 [ 49.09929657   1.        ]
 [ 54.11745834

In [71]:
y_pred = (model.predict(X_test))
#model only predicts 1s
y_pred

array([[ 98.689766],
       [ 99.12061 ],
       [ 99.12061 ],
       [ 81.15881 ],
       [101.59487 ],
       [ 98.01075 ],
       [ 53.2935  ],
       [ 80.060905],
       [100.56113 ],
       [100.56113 ],
       [100.24984 ],
       [100.70218 ],
       [ 98.75565 ],
       [ 98.75565 ],
       [ 80.29125 ],
       [ 80.29125 ],
       [ 80.29125 ],
       [ 84.227394],
       [ 84.227394],
       [ 84.227394],
       [102.7243  ],
       [ 80.56621 ],
       [ 80.56621 ],
       [102.001564],
       [100.223755],
       [100.223755],
       [100.87411 ],
       [118.2699  ],
       [122.1925  ],
       [130.39545 ],
       [204.43558 ],
       [186.89363 ],
       [237.32799 ],
       [226.7531  ],
       [226.7531  ],
       [228.4608  ],
       [238.68639 ],
       [238.68639 ],
       [235.06717 ],
       [421.1721  ],
       [386.37198 ],
       [444.08255 ],
       [444.08255 ],
       [228.47356 ],
       [231.88063 ],
       [201.08833 ],
       [228.4608  ],
       [423.6

In [76]:
y_test.shape

(166,)

In [77]:
y_pred.shape

(166, 1)

In [78]:
y_test[0]

96.2986526489257

In [80]:
model.predict(X_test)[0]

array([98.689766], dtype=float32)

Result: This model did also not converge, meaning that its loss is too high to create usable results.